In [ ]:
from google.colab import files
files.upload()


In [ ]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [ ]:
!kaggle datasets list


In [ ]:
!pip install -q kaggle

In [ ]:
!kaggle datasets download -d jtiptj/chest-xray-pneumoniacovid19tuberculosis

In [ ]:
!ls

In [ ]:
!unzip chest-xray-pneumoniacovid19tuberculosis.zip

In [ ]:
!ls

In [ ]:

!mv train/TURBERCULOSIS train/TUBERCULOSIS
!mv val/TURBERCULOSIS val/TUBERCULOSIS
!mv test/TURBERCULOSIS test/TUBERCULOSIS


In [ ]:
!ls train
!ls val
!ls test

In [ ]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

train_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

eval_tfms = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
])

train_ds = datasets.ImageFolder("/content/train", transform=train_tfms)
val_ds   = datasets.ImageFolder("/content/val",   transform=eval_tfms)
test_ds  = datasets.ImageFolder("/content/test",  transform=eval_tfms)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_ds,   batch_size=16, num_workers=0)
test_loader  = DataLoader(test_ds,  batch_size=16, num_workers=0)

print("Classes:", train_ds.class_to_idx)


In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import timm
import torch.nn as nn

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

In [ ]:
!ls /content

In [ ]:
def train_epoch():
    model.train()
    total = 0
    for x,y in train_loader:
        x,y = x.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x)
        loss = criterion(out,y)
        loss.backward()
        optimizer.step()
        total += loss.item()
    return total/len(train_loader)

def eval_epoch(loader):
    model.eval()
    correct=total=0
    with torch.no_grad():
        for x,y in loader:
            x,y = x.to(device), y.to(device)
            _,p = model(x).max(1)
            total += y.size(0)
            correct += (p==y).sum().item()
    return correct/total


In [ ]:

model = timm.create_model(
    "densenet121",
    pretrained=True,
    num_classes=4
)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print("Model defined and moved to device")

In [ ]:
print(type(model))

In [ ]:
best = 0.0

for epoch in range(1, 11):
    model.train()
    running_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    val_acc = eval_epoch(val_loader)
    print(f"Epoch {epoch} | Loss {running_loss/len(train_loader):.4f} | Val Acc {val_acc:.4f}")

    if val_acc > best:
        best = val_acc
        torch.save(model.state_dict(), "best_densenet.pth")


In [ ]:
 model.load_state_dict(torch.load("best_densenet.pth"))
test_acc = eval_epoch(test_loader)
print("Final Test Accuracy:", test_acc)



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CustomCNN(nn.Module):
    def __init__(self, num_classes=4):
        super(CustomCNN, self).__init__()

        # Block 1
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1   = nn.BatchNorm2d(32)

        # Block 2
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2   = nn.BatchNorm2d(64)

        # Block 3
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3   = nn.BatchNorm2d(128)

        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.5)

        # After 3 pools: 224 → 112 → 56 → 28
        self.fc1 = nn.Linear(128 * 28 * 28, 256)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))

        x = x.view(x.size(0), -1)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)

        return x

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CustomCNN(num_classes=4).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

print(model)

In [ ]:
def train_epoch():
    model.train()
    total_loss = 0.0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(train_loader)


def eval_epoch(loader):
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct / total


In [ ]:
best_acc = 0.0

for epoch in range(1, 16):
    loss = train_epoch()
    val_acc = eval_epoch(val_loader)

    print(f"Epoch {epoch} | Loss: {loss:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), "best_custom_cnn.pth")

In [ ]:
model.load_state_dict(torch.load("best_custom_cnn.pth"))
test_acc = eval_epoch(test_loader)
print("Custom CNN Test Accuracy:", test_acc)

In [ ]:
import torch

def evaluate_model(model, loader):
    model.eval()
    correct = total = 0

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            total += labels.size(0)
            correct += (preds == labels).sum().item()

    return correct / total

In [ ]:
# Load DenseNet
densenet = timm.create_model(
    "densenet121",
    pretrained=False,
    num_classes=4
).to(device)

densenet.load_state_dict(torch.load("best_densenet.pth"))

# Load Custom CNN
custom_cnn = CustomCNN(num_classes=4).to(device)
custom_cnn.load_state_dict(torch.load("best_custom_cnn.pth"))

# Evaluate
densenet_acc = evaluate_model(densenet, test_loader)
cnn_acc = evaluate_model(custom_cnn, test_loader)

print(f"DenseNet Test Accuracy: {densenet_acc:.4f}")
print(f"Custom CNN Test Accuracy: {cnn_acc:.4f}")

In [ ]:
best_model = densenet if densenet_acc >= cnn_acc else custom_cnn
best_model_name = "DenseNet-121" if densenet_acc >= cnn_acc else "Custom CNN"

print("Best Model:", best_model_name)

In [ ]:
torch.save(best_model.state_dict(), "final_best_model.pth")

In [ ]:
!pip install -q gradio pillow

In [ ]:
!pip install -q gradio pillow opencv-python

In [ ]:

import torch
import timm
import gradio as gr
import numpy as np
import cv2
from PIL import Image
from torchvision import transforms

# ---------------- CONFIG ----------------
CLASSES = ["COVID19", "NORMAL", "PNEUMONIA", "TUBERCULOSIS"]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ---------------- LOAD MODEL ----------------
model = timm.create_model("densenet121", pretrained=False, num_classes=4)
model.load_state_dict(torch.load("final_best_model.pth", map_location=device))
model.to(device)
model.eval()

# ---------------- TRANSFORM ----------------
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

# ---------------- GRAD-CAM FIXED ----------------
class GradCAM:
    def __init__(self, model, target_layer):
        self.model = model
        self.gradients = None
        self.activations = None

        target_layer.register_forward_hook(self.save_activation)
        target_layer.register_full_backward_hook(self.save_gradient)

    def save_activation(self, module, inp, out):
        self.activations = out.detach()

    def save_gradient(self, module, grad_input, grad_output):
        self.gradients = grad_output[0].detach()

    def generate(self):
        weights = self.gradients.mean(dim=(2,3), keepdim=True)
        cam = (weights * self.activations).sum(dim=1)
        cam = torch.relu(cam)
        cam = cam[0].cpu().numpy()
        cam = (cam - cam.min()) / (cam.max() - cam.min() + 1e-8)
        return cam

# Last conv layer of DenseNet
gradcam = GradCAM(model, model.features[-1])

# ---------------- PREDICT FUNCTION ----------------
def predict_xray(image):
    try:
        image = image.convert("RGB")
        orig = np.array(image.resize((224,224)))

        img = transform(image).unsqueeze(0).to(device)
        output = model(img)
        probs = torch.softmax(output, dim=1)

        pred = torch.argmax(probs, dim=1).item()

        model.zero_grad()
        output[0, pred].backward()

        cam = gradcam.generate()
        cam = cv2.resize(cam, (224,224))
        heatmap = cv2.applyColorMap(np.uint8(255 * cam), cv2.COLORMAP_JET)
        overlay = cv2.addWeighted(orig, 0.6, heatmap, 0.4, 0)

        return (
            CLASSES[pred],
            f"{probs[0][pred].item()*100:.2f}%",
            Image.fromarray(overlay)
        )

    except Exception as e:
        return "Error", "Error", None

# ---------------- GUI (UPDATED) ----------------
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown(
        """
        # 🩺 Chest X-ray Disease Detection (Explainable AI)
        Upload a chest X-ray image to detect the disease and visualize
        **infected lung regions using Grad-CAM**.
        """
    )

    with gr.Row():
        with gr.Column(scale=1):
            xray_input = gr.Image(
                type="pil",
                label="Upload Chest X-ray",
                height=300
            )
            predict_btn = gr.Button("🔍 Analyze X-ray")

        with gr.Column(scale=1):
            disease_output = gr.Textbox(
                label="Predicted Disease",
                interactive=False
            )
            confidence_output = gr.Textbox(
                label="Confidence",
                interactive=False
            )
            heatmap_output = gr.Image(
                label="Grad-CAM Heatmap",
                height=300
            )

    predict_btn.click(
        fn=predict_xray,
        inputs=xray_input,
        outputs=[disease_output, confidence_output, heatmap_output]
    )

demo.launch(share=True)

